In [1]:
import numpy as np

from sb3_contrib import RecurrentPPO
from stable_baselines3.common.evaluation import evaluate_policy
from sb3_contrib.common.recurrent.type_aliases import RNNStates
import gym
import torch as th
from active_critic.utils.gym_utils import make_dummy_vec_env, sample_expert_transitions_rollouts, parse_sampled_transitions, DummyExtractor
from active_critic.utils.pytorch_utils import calcMSE


/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [2]:
from active_critic.utils.gym_utils import make_policy_dict

In [3]:
env_tag = 'pickplace'
seq_len = 100
n_demonstrations = 10
env, vec_expert = make_dummy_vec_env(name=env_tag, seq_len=seq_len)
env, _ = make_dummy_vec_env(name=env_tag, seq_len=seq_len)
transitions, rollouts = sample_expert_transitions_rollouts(
    vec_expert.predict, env, n_demonstrations)
actions, observations, rewards = parse_sampled_transitions(transitions=transitions, extractor=DummyExtractor(), seq_len=seq_len, device='cuda')
model = RecurrentPPO("MlpLstmPolicy", env=env, verbose=1)
lstm_states = model._last_lstm_states
a = model.policy.train()

/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/metaworld/policies/policy.py:41: UserWarning: Constant(s) may be too high. Environments clip response to [-1, 1]
  warnings.warn('Constant(s) may be too high. Environments clip response to [-1, 1]')


Using cuda device


/home/hendrik/Documents/master_project/Code/active_critic/src/active_critic/utils/gym_utils.py:268: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  actions = th.tensor(fill_arrays(actions, seq_len = seq_len), dtype=th.float, device=device)


In [4]:
observations.shape

torch.Size([10, 100, 39])

In [5]:
inpt_obsv = observations[:,:1].repeat([1, observations.shape[1], 1])

In [6]:
#inpt_obsv = inpt_obsv.transpose(0,1)

In [7]:
inpt_obsv.shape

torch.Size([10, 100, 39])

In [8]:
# = actions.transpose(0,1)

In [13]:
batch_size = inpt_obsv.shape[0]
lstm = model.policy.lstm_actor
a = model.policy.train()

single_hidden_state_shape = (lstm.num_layers, batch_size, lstm.hidden_size)
# hidden and cell states for actor and critic
lstm_states = RNNStates(
    (
        th.zeros(single_hidden_state_shape).to(model.device),
        th.zeros(single_hidden_state_shape).to(model.device),
    ),
    (
        th.zeros(single_hidden_state_shape).to(model.device),
        th.zeros(single_hidden_state_shape).to(model.device),
    ),
)

for i in range(20000):
    rnn_states_list = [lstm_states]
    actions_res, values, log_prob, rnn_states = model.policy.forward(obs = inpt_obsv, lstm_states=lstm_states, episode_starts = th.zeros([inpt_obsv.shape[0] * inpt_obsv.shape[1]], device='cuda'), deterministic=False)
    model.policy.optimizer.zero_grad()
    loss = calcMSE(actions_res.reshape(-1), actions.reshape(-1))
    loss.backward()
    model.policy.optimizer.step()
    if i % 2000 == 0:
        print(loss)


tensor(1.5340, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.3898, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.1845, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0890, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0454, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0230, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0110, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0057, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0030, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0015, device='cuda:0', grad_fn=<MeanBackward0>)


In [14]:
all_actions = []
lstm_states = None
for i in range(observations.shape[1]):
    if i == 0:
        episode_start_c = 1
    else:
        episode_start_c = 0
    pred_actions, lstm_states = model.predict(observation=inpt_obsv[0,:1].cpu().numpy(), deterministic=True, episode_start=np.array([episode_start_c]), state=lstm_states)
    all_actions.append(pred_actions)

In [15]:
all_actions = np.array(all_actions)

In [16]:
wanted_actions = actions[0].cpu().numpy()

In [17]:
diff = ((all_actions.reshape([-1]) - wanted_actions.reshape([-1]))**2).mean()
diff

0.00072560186

In [22]:
pred_actions, rnn_states = model.predict(observation=inpt_obsv[0,:1].cpu().numpy(), deterministic=True, episode_start=np.array([0]), state=rnn_states)

In [23]:
pred_actions

array([[ 0.84174454,  0.05192827, -0.75935364, -0.03078466]],
      dtype=float32)

In [21]:
actions[0,:1]

tensor([[ 0.7563,  0.0587, -0.7430,  0.0000]], device='cuda:0')

In [13]:
actions_res.shape

torch.Size([1000, 4])

In [14]:
actions_res.reshape([2, 20,4])

RuntimeError: shape '[2, 20, 4]' is invalid for input of size 4000

In [ ]:
for j in range(100000):
    rnn_states_list = [RNNStates(pi=lstm_states[0], vf=lstm_states[1])]

    for i in range(actions.shape[1]):
        _, values, log_prob, rnn_states = model.policy.forward(obs = obs, lstm_states=rnn_states_list[-1], episode_starts = th.zeros([1], device='cuda'), deterministic=False)
        rnn_states_list.append(rnn_states)

    model.policy.optimizer.zero_grad()
    total_loss = 0
    for i in range(len(rnn_states_list) - 1):
        if i == 0:
            episode_starts = th.ones([1], device='cuda')
        else:
            episode_starts = th.zeros([1], device='cuda')

        wanted_action = actions[:,i]

        value, log_prob, entropy = model.policy.evaluate_actions(obs, wanted_action, rnn_states_list[i], episode_starts = episode_starts)
        prob_true_act = th.exp(log_prob).mean()
        log_prob = log_prob.mean()
        entropy = entropy.mean()


        ent_loss = 0 * entropy
        neglogp = -log_prob
        loss = neglogp + ent_loss
        total_loss = total_loss + loss
    total_loss = total_loss.mean()
    if j%100 == 0:
        print(f'total_loss {total_loss}')
    total_loss.backward()
    model.policy.optimizer.step()


In [ ]:
rnn_actions = []
rnn_states_list = [RNNStates(pi=lstm_states[0], vf=lstm_states[1])]
for i in range(actions.shape[1]):
    rnn_action, values, log_prob, rnn_states = model.policy.forward(obs = obs, lstm_states=rnn_states_list[-1], episode_starts = th.zeros([1], device='cuda'), deterministic=True)
    rnn_actions.append(rnn_action)    
    rnn_states_list.append(rnn_states)


In [ ]:
loss = 0
for i, rnn_action in enumerate(rnn_actions):
    loss += calcMSE(rnn_action, actions[:,i])

In [ ]:
loss

In [ ]:
actions

In [ ]:
o, states = model.policy.predict(obs.cpu().numpy())

In [ ]:
o

In [ ]:
actions.shape

In [ ]:
o, states = model.policy.predict(obs.cpu().numpy(), state=states)

In [ ]:
o

In [ ]:
o, states = model.policy.predict(obs.cpu().numpy(), state=states)

In [ ]:
o

In [ ]:
loss.mean().backward()

In [ ]:
rnn_states_list[-1][0][0].mean().backward()

In [ ]:
model.policy.lstm_actor.all_weights[0][0]._grad

In [ ]:
ent_weight = 1e-2

eval actions:         
obs: th.Tensor,
actions: th.Tensor,
lstm_states: RNNStates,
episode_starts: th.Tensor,

-> 
values, log_prob, distribution.entropy()

In [ ]:
_, log_prob, entropy = model.policy.evaluate_actions(obs, actions, rnn_states, episode_starts = th.ones([1], device='cuda'))
#print(f'max log: {log_prob.max()}')

prob_true_act = th.exp(log_prob).mean()
log_prob = log_prob.mean()
entropy = entropy.mean()


ent_loss = ent_weight * entropy
neglogp = -log_prob
loss = neglogp + ent_loss

In [ ]:
loss

In [ ]:
model.policy.lstm_actor.forward(obs = env.reset(), lstm_states=lstm_states, epsidode_starts = th.zeros([1]))

In [ ]:
from imitation.algorithms.bc import BC
from active_critic.utils.gym_utils import (
    make_vec_env,
    make_dummy_vec_env,
    sample_expert_transitions_rollouts,
    make_pomdp_rollouts,
    make_dummy_vec_env_pomdp,
    get_avr_succ_rew_det
)

In [ ]:
env_tag = 'pickaplce'
seq_len = 100
n_demonstrations

env, vec_expert = make_dummy_vec_env(name=env_tag, seq_len=seq_len)
val_env, _ = make_dummy_vec_env(name=env_tag, seq_len=seq_len)

transitions, rollouts = sample_expert_transitions_rollouts(
    vec_expert.predict, val_env, n_demonstrations)